In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import polars as pl
import matplotlib.pyplot as plt
import ee
# import zipfile
# import os
# import request
import rasterio
import geeS2downloader
from IPython.display import Image

In [4]:
# Initialize GEE API
ee.Initialize()

In [5]:
# Parameters for collection

# Point of interest (Center of Aguascalientes City) EPGS: 
lat = 21.8833333
lon = -102.3

# Punto de interes
poi = ee.Geometry.Point(lon, lat)# 10 km buffer 
aoi = ee.Geometry.LinearRing([[-102.35, 21.9], 
                              [-102.23, 21.9], 
                              [-102.23, 21.8], 
                              [-102.35, 21.8]])

# Periodo de tiempo
start_date = "2022-12-10"
end_date = "2022-12-15"

In [6]:
# Image collection
ags_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
                          .filterBounds(poi)\
                          .filterDate(ee.Date(start_date), ee.Date(end_date))
print("Total images:", ags_collection.size().getInfo())

# Add latitude and longitude to the collection
# ags_collection = ags_collection.addBands(ee.Image.pixelLonLat())
ags_list = ags_collection.toList(ags_collection.size())

Total images: 2


In [22]:
display(Image(url = ee.Image(ags_list.get(0)).clip(poi.buffer(buff_size).bounds(proj = "EPSG:32613", maxError = 0.1)).getThumbURL({"min": 0,
                                                "max": 3000,
                                                "dimensions": 800,
                                                "bands": ["B4", "B3", "B2"]})))


In [34]:
# Define bands and buffer
bands = ["B4", "B3", "B2"]
buff_size = 1000

In [37]:
# Save first image and clip with a rectangle buffer
img0 = ee.Image(ags_list.get(0)) # Raw image
# img0 = img0.addBands(ee.Image.pixelLonLat())
img0 = img0.select(bands) # Select bands
img0 = img0.clip(poi.buffer(buff_size).bounds(proj = "EPSG:32613", maxError = 0.1)) # Rectangle buffer

In [39]:
# Download link
link = img0.getDownloadURL(
    {
    "scale": 10,
    "crs": "EPSG:32613",
    "fileFormat": "GeoTIFF",
    "maxPixels": 1e13
    }
)

print(link)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/680bfa8072c7ecb2806faf62aa4b6359-5edab17fa6a96f6400750f302accc532:getPixels


## Download GeoTIFFs

In [40]:
# Bands to download
bands = ["B4", "B3", "B2", "B1", "B5", "B6", "B7", "B8", "B8A", "B9", "B11", "B12", "WVP"]

In [10]:
import geemap
import ee
from geeS2downloader import GEES2Downloader

# If it is not yet authenticated, uncomment the line below
# ee.Authenticate()

ee.Initialize()

img = ags_collection

Map = geemap.Map(center=(23, -102.4), zoom=9)
Map.addLayer(img, {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', "B2"], "region": poi.buffer(10000)}, 'Cloud Probability', True, 1)
Map

Map(center=[23, -102.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

In [15]:
# Export with geemap
geemap.ee_export_image(cld, "../data/images/clouds.tif", scale=10)

Generating URL ...
An error occurred while downloading.
Total request size (241208648 bytes) must be less than or equal to 50331648 bytes.


In [6]:
# Export with GEE2Downloader
downloader = GEES2Downloader()

downloader.download(img = cld, band = "probability")

Retrieving band info
Dividing band in 16 tiles


Tiles:   0%|          | 0/16 [00:00<?, ?tile/s]

Tile[0:2896,8688:10980]:   0%|          | 0.00/8.63M [00:00<?, ?b/s]

Tile[0:2896,2896:5792]:   0%|          | 0.00/10.9M [00:00<?, ?b/s]

Tile[0:2896,5792:8688]:   0%|          | 0.00/10.9M [00:00<?, ?b/s]

...

Tile[8688:10980,2896:5792]:   0%|          | 0.00/8.63M [00:00<?, ?b/s]

Tile[8688:10980,8688:10980]:   0%|          | 0.00/6.83M [00:00<?, ?b/s]

Tile[8688:10980,5792:8688]:   0%|          | 0.00/8.63M [00:00<?, ?b/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "rasterio\_io.pyx", line 701, in rasterio._io.DatasetReaderBase._read
  File "rasterio\shim_rasterioex.pxi", line 162, in rasterio._shim.io_multi_band
  File "rasterio\_err.pyx", line 193, in rasterio._err.exc_wrap_int
rasterio._err.CPLE_AppDefinedError: /vsimem/f6cf55cb-2618-41a3-a054-318784b6d071/f6cf55cb-2618-41a3-a054-318784b6d071.tif, band 1: IReadBlock failed at X offset 0, Y offset 0: TIFFReadEncodedTile() failed.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Erick\anaconda3\envs\rs_leaks_env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Erick\AppData\Local\Temp\ipykernel_3656\2843627139.py", line 4, in <module>
    downloader.download(img = cld, band = "probability")
  File "c:\Users\Erick\anaconda3\envs\rs_leaks_env\lib\site-packages\geeS2downloader\gee.py", lin

In [18]:
print(downloader.array)

None
